In [1]:
import backtester as bt
import pandas as pd

In [2]:
df = bt.feed.CSVData(
	files=[
		"/Users/pol.maksim/Downloads/OHLC Data/binance/futures/BTCUSDT/1m/2021.csv",
		"/Users/pol.maksim/Downloads/OHLC Data/binance/futures/BTCUSDT/1m/2022.csv"
	],
	timestamp=0,
	datetime=1,
	open=3,
	high=4,
	low=5,
	close=6,
	volume=7,
	dtformat="%Y-%m-%d %H:%M:%S",
	separator=",",
	header=1,
	start_date=pd.Timestamp(year=2017, month=1, day=1),
	# end_date=pd.Timestamp(year=2017, month=12, day=31)
).read()

In [3]:
class BuyAndHold24Hours(bt.Strategy):
	def __init__(self):
		super().__init__()

		self.min_trade_notional = 10
		self.max_balance_risk = 0.1

	def next(self):
		if self.last.datetime.hour == 0 and self.last.datetime.minute == 0:
			notional = self.broker.cash * self.max_balance_risk

			if notional >= self.min_trade_notional:
				qty = notional / self.last.close
				self.open_long(quantity=qty)
		elif self.last.datetime.hour == 23 and self.last.datetime.minute == 59:
			if self.has_long:
				self.close_long()

In [4]:
strategy = BuyAndHold24Hours()
strategy.set_fee_rate(0.04)
strategy.set_funding_rate(0.01)
strategy.set_base_precision(8)
strategy.set_quote_precision(2)
strategy.set_price_precision(2)
strategy.set_cash(10000)
strategy.set_data(df)

In [5]:
report = strategy.run()

In [ ]:
report.plot()